In [1]:
from ctypes import *
import imageio
import numpy as np
import matplotlib.pyplot as plt


In [2]:
cdll.LoadLibrary("Blink_SDK_C.dll")
slm_lib = CDLL("Blink_SDK_C")

In [3]:
LUT = 'slm_h2_encrypt_noPhaseWrap.txt'
blank_image = "512white.bmp"

LUT = bytes(LUT, 'utf-8')
lut_file = c_char_p(LUT)

create_SDK_func = slm_lib.Create_SDK
create_SDK_func.argtypes = (c_uint, c_uint, POINTER(c_uint), POINTER(c_bool), c_bool, c_bool, c_bool, c_uint, c_char_p)
# in python 3 the integer returned by create_SDK_func is bigger than 32 bit, so had to reassign the integer type to longlong
create_SDK_func.restype = c_longlong

# Basic parameters for calling Create_SDK
slm_resolution = c_uint(512)
# currently only supports 8 bit depth
bit_depth = c_uint(8)
num_boards_found = c_uint(0)
constructed_okay = c_bool(0)
is_nematic_type = c_bool(1)
RAM_write_enable = c_bool(1)
use_GPU = c_bool(1)
max_transients = c_uint(20)
true_frames = c_int(3)

# Call the Create_SDK constructor
# Returns a handle that's passed to subsequent SDK calls
sdk = create_SDK_func(bit_depth, slm_resolution, byref(num_boards_found), 
                     byref(constructed_okay), is_nematic_type, 
                     RAM_write_enable, use_GPU, max_transients,lut_file)


In [12]:
test_mask = np.ones((512,512), dtype= 'uint8')
n_bytes = c_uint(100)

In [16]:
#bool Calculate_transient_frames(Blink_SDK *sdk, const unsigned char* target_phase, unsigned int* byte_count);

Calculate_transient_frames_func = slm_lib.Calculate_transient_frames

Calculate_transient_frames_func.argtypes = (c_longlong, POINTER(c_ubyte), POINTER(c_uint))
Calculate_transient_frames_func.restype = c_bool

ctf_result = Calculate_transient_frames_func(sdk,test_mask.ctypes.data_as(POINTER(c_ubyte)), byref(n_bytes))
ctf_result

False